In [1]:
import pandas as pd
import altair as alt
import geojson
import os
from urllib.request import urlopen

In [2]:
df = pd.read_csv('/kaggle/input/canada-wildfires-1930-2021/NFDB_point_20220901.txt', parse_dates=[9,10,11])

/tmp/ipykernel_21/2662470203.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/canada-wildfires-1930-2021/NFDB_point_20220901.txt', parse_dates=[9,10,11])


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423831 entries, 0 to 423830
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   FID         423831 non-null  int64         
 1   SRC_AGENCY  423831 non-null  object        
 2   FIRE_ID     423666 non-null  object        
 3   FIRENAME    423830 non-null  object        
 4   LATITUDE    423831 non-null  float64       
 5   LONGITUDE   423831 non-null  float64       
 6   YEAR        423831 non-null  int64         
 7   MONTH       423831 non-null  int64         
 8   DAY         423831 non-null  int64         
 9   REP_DATE    420118 non-null  datetime64[ns]
 10  ATTK_DATE   28740 non-null   datetime64[ns]
 11  OUT_DATE    213092 non-null  datetime64[ns]
 12  DECADE      423831 non-null  object        
 13  SIZE_HA     423831 non-null  float64       
 14  CAUSE       423590 non-null  object        
 15  PROTZONE    422821 non-null  object        
 16  FI

In [4]:
df.describe(include='all', datetime_is_numeric=True).transpose()

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
FID,423831.0,NaN,NaN,NaN,211915.0,0.0,105957.5,211915.0,317872.5,423830.0,122349.615308
SRC_AGENCY,423831,56,BC,149410,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FIRE_ID,423666,320953,COC1,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FIRENAME,423830,25721,,362543,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LATITUDE,423831.0,NaN,NaN,NaN,51.895936,0.0,49.1337,51.023,54.630289,70.0,4.46777
LONGITUDE,423831.0,NaN,NaN,NaN,-104.253547,-166.044,-120.11235,-114.503,-88.303603,116.188,20.412661
YEAR,423831.0,NaN,NaN,NaN,1990.464784,-999.0,1979.0,1991.0,2005.0,2021.0,47.872423
MONTH,423831.0,NaN,NaN,NaN,6.585818,0.0,5.0,7.0,8.0,12.0,1.675279
DAY,423831.0,NaN,NaN,NaN,15.441048,0.0,8.0,15.0,23.0,31.0,8.98398
REP_DATE,420118,NaN,NaN,NaN,1991-07-30 22:55:33.657686656,1930-03-07 00:00:00,1979-07-28 00:00:00,1991-07-04 00:00:00,2005-06-29 00:00:00,2021-12-14 00:00:00,NaN


In [5]:
df = df[df.LATITUDE > 41.6]
df_subset = df.sample(5000, random_state=42)

In [6]:
df.DECADE.value_counts().sort_index()

1930-1939        1
1940-1949       75
1950-1959    16634
1960-1969    30426
1970-1979    61208
1980-1989    87446
1990-1999    83936
2000-2009    73127
2010-2019    60121
2020-2029    10705
Name: DECADE, dtype: int64

In [7]:
url = 'https://raw.githubusercontent.com/smaslo2/Wildfires/main/canada.geojson'
response = urlopen(url)
data_file = geojson.load(response)
# with open('canada.geojson') as f:
#   data_file = geojson.load(f)
data_geojson = alt.InlineData(values=data_file, format=alt.DataFormat(property='features',type='json'))

In [8]:
selection = alt.selection_point(fields=['DECADE'], bind='legend')

background = alt.Chart(data_geojson).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    type='identity',
    reflectY=True
).properties(
    width=500,
    height=300
)

points = alt.Chart(df_subset).mark_circle().encode(
    latitude='LATITUDE', 
    longitude='LONGITUDE', 
    size=alt.value(7),
    color='DECADE',
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection)

time = alt.Chart(df_subset).mark_line().properties(
    width=300,
    height=200
).encode(
    x='MONTH',
    y=alt.Y(aggregate='count'),
    color='DECADE',
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection)

decades = alt.Chart(df_subset).mark_bar().properties(
    width=200,
    height=200
).encode(
    x='DECADE',
    y=alt.Y(aggregate='count'),
    color='DECADE',
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection)

footnote = alt.Chart(
    {"values": [{"text": "Canadian Forest Service. 2010. National Fire Database - Agency Fire Data. Natural Resources Canada, Canadian Forest Service, Northern Forestry Centre, Edmonton, Alberta."}]}
).mark_text(size=10, lineBreak=r'\n').encode(
    text="text:N"
).properties(width=500)

title = alt.Chart(
    {"values": [{"text": "Canadian Wildfires 1930-2021 (Randomly Sampled)"}]}
).mark_text(size=30, lineBreak=r'\n').encode(
    text="text:N"
).properties(width=500)

alt.vconcat(title, alt.vconcat(background+points, time | decades), footnote)

alt.VConcatChart(...)